# Modelagem de Dispersão

Neste notebook vamos implementar um modelo de dispersão gaussiano

In [1]:
#Criando uma função do modelo gaussiano
import numpy as np

def modeloGaussiano(qs, sigmaY, sigmaZ, u, y, z, H): # def usa para chamar novamente
    termo1 = qs/(2*np.pi*sigmaY*sigmaZ*u)
    termo2 = np.exp((-y**2)/(2*sigmaY**2))
    termo3 = np.exp((-(z-H)**2)/(2*sigmaZ**2)) + np.exp((-(z+H)**2)/(2*sigmaZ**2))
    conc = termo1*termo2*termo3 #conc é a concentração
    return conc


In [5]:
# Utilizando a função do modelo Gaussiano
qs = 100 #taxa de emissão em g/s
sigmaY = 10 #espalhamento vertical em m
sigmaZ = 15 #espalhamento horizontal em m
u = 10 #velocidade vento
y = 0 #pra ser na direção do vento, usa y=0
z = 1.5 #altura media do nariz
H = 50 #altura que a pluma subiu

conc = modeloGaussiano (qs, sigmaY, sigmaZ, u, y, z, H)
conc = conc*10**6 #muda a unidade de medida de g/s para µg/m³
print ("A concentração estimada é = " +str(conc)+ 'µg/m³')

A concentração estimada é = 86.2052564132534µg/m³
